In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()

import math
import numpy as np
import itertools
import collections


In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
# get all segment file names and save in Filenames
def scantext():        
    x=[]
    for path,dirs,files in os.walk('./waymo-od/data'):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)



Filenames=scantext()

In [ ]:
### Extract segment files in Frames, each file consists of multiple frames

Frames = []
for file in Filenames[0:1]:
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        Frames.append(frame)

## Each frame includes:
#### 1. ***context***: similar for all the frames in one segment
#### 2. ***timestamp_micros*** : should be devided to 1000000
#### 3. ***pose***
#### 4. ***laser_labels*** : includes
#### 5. ***camera_labels***

In [44]:
def CountLLabels(frame):    #number of laser labels in a frame
    return len(frame.laser_labels)



In [104]:
pedLoc = collections.defaultdict(list)
pedMeta = collections.defaultdict(list)
for i in range(len(Frames)):
    t = (Frames[i].timestamp_micros/1000000)                       #time
    Nlabels = CountLLabels(Frames[i])                              # number of laser labels
    for j in range(Nlabels):
        
        if Frames[i].laser_labels[j].type == 2:                    # type 2: pedestrian
            PedID = Frames[i].laser_labels[j].id
            
            pedLoc[PedID].append(t)
            pedLoc[PedID].append(Frames[i].laser_labels[j].box)
            
            pedMeta[PedID].append(t)
            pedMeta[PedID].append(Frames[i].laser_labels[j].metadata)

                
                    
            
            

397